In [4]:
%load_ext autoreload
%autoreload 2

import torch
torch.set_default_device('cuda') 

import os.path
from pathlib import Path
import uuid

sess_id = str(uuid.uuid4())[:8]
sess_path = Path(f'session_{sess_id}')
sess_path.mkdir(exist_ok=True)

states_path = Path('states')
states_path.mkdir(exist_ok=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from emulator import Emulator, SIMPLE_ACTION_SPACE, MOVEMENT_ACTION_SPACE
from pokered_vecenv import PokeRedVecEnv

from explore_low_agent import ExploreLowAgent
from basic_flee_agent import BasicFleeAgent

from policy import Policy

gb_path = './PokemonRed.gb'
init_state = './has_pokedex_nballs.state'
init_state = 'states/11_5_0-0.state'

emulators = [Emulator(sess_path, gb_path, instance_id=f'main_{i}', headless=True) for i in range(10)]
vec_env = PokeRedVecEnv(emulators, [init_state])

flee_policy = Policy([], SIMPLE_ACTION_SPACE)
flee_agent = BasicFleeAgent('flee_agent', SIMPLE_ACTION_SPACE)

explore_policy = Policy([], MOVEMENT_ACTION_SPACE)
explore_agent = ExploreLowAgent('explore_agent', MOVEMENT_ACTION_SPACE, 12)

vec_env.register_agent(flee_agent)
vec_env.register_agent(explore_agent)
vec_env.initial_agent('explore_agent')

policies = {'explore_agent': explore_policy, 'flee_agent': flee_policy}

In [6]:
from ppo import PPOSettings, PPOTrainer
pposettings = PPOSettings()
ppo = PPOTrainer(policies['explore_agent'], pposettings)

g:\Anaconda3\envs\pokey\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from training import train

#temp variables
num_envs = 10
num_steps = 400
num_iterations = 100

train(policies, vec_env, num_envs, 'explore_agent', ppo, num_steps=num_steps, num_iterations=num_iterations, verbose=True)